# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [479]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import json
import pandas as pd
from decimal import Decimal

In [480]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [481]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'uk_food']


In [482]:
# assign the uk_food database to a variable name
db = mongo['uk_food']
collection = db['establishments']

In [483]:
# review the collections in our new database
with open('Resources/establishments.json', 'r') as file:
    data = json.load(file)
    collection.insert_many(data)

In [484]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [485]:
# review a document in the establishments collection
print(db.establishments.find_one())

{'_id': ObjectId('64ff8c820f902f894106c3ac'), 'FHRSID': 254719, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000069980', 'BusinessName': 'Refreshment Kiosk', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'The Bay', 'AddressLine2': 'St Margarets Bay', 'AddressLine3': 'Kent', 'AddressLine4': '', 'PostCode': 'CT15 6DY', 'Phone': '', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2022-03-24T00:00:00', 'LocalAuthorityCode': '182', 'LocalAuthorityName': 'Dover', 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/', 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk', 'scores': {'Hygiene': 0, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.387974', 'latitude': '51.152225'}, 'RightToReply': '', 'Distance': 4587.347174863443, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode': None, 'totalCount': 0, 'total

In [486]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [487]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "name": "Penang Flavours",
    "address": {
        "street": "123 Elm Street",
        "city": "Greenwich",
        "postcode": "SE10 9XYZ",
    },
    "cuisine": "Halal",
    "rating": None
}

In [488]:
# Insert the new restaurant into the collection
insert_restaurant = collection.insert_one(new_restaurant)


In [489]:
# Check that the new restaurant was inserted
print(establishments)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'uk_food'), 'establishments')


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [490]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
projection = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [491]:
# Update the new restaurant with the correct BusinessTypeID
result = collection.find_one(query, projection)

In [492]:
# Confirm that the new restaurant was updated
if result:
    print("BusinessTypeID and BusinessType for 'Restaurant/Cafe/Canteen':")
    print(result)
else:
    print("No matching document found.")

BusinessTypeID and BusinessType for 'Restaurant/Cafe/Canteen':
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [493]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}
count = collection.count_documents(query)

In [494]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = collection.delete_many(query)

In [495]:
# Check if any remaining documents include Dover
if count > 0:
    print("There are remaining documents that include 'Dover' in LocalAuthorityName.")
else:
    print("No remaining documents include 'Dover' in LocalAuthorityName.")

There are remaining documents that include 'Dover' in LocalAuthorityName.


In [502]:
# Check that other documents remain with 'find_one'
other_documents = collection.find_one(query)

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [507]:
# Change the data type from String to Decimal for longitude and latitude
filter_query = {
    "latitude": {"$type": "string"},
    "longitude": {"$type": "string"}
}

update_result = collection.update_many(
    filter_query,
    {
        "$set": {
            "latitude": {"$Decimal": "$latitude"},
            "longitude": {"$Decimal": "$longitude"}
        }
    }
)

Use `update_many` to convert `RatingValue` to integer numbers.

In [511]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [513]:
# Change the data type from String to Integer for RatingValue
update_result = collection.update_many(
    {"RatingValue": {"$exists": True}},
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)


In [514]:
# Check that the coordinates and rating value are now numbers
df.dtypes

Series([], dtype: object)